In [1]:
# 🔄 RESTART KERNEL FIRST to clear all cached state!
# This cell tests the basic foreign key functionality without relationships

from datetime import date as Date
from sqlmodel import SQLModel, create_engine, Session, select, Field
from typing import Optional

# Define minimal test models inline to avoid import caching issues
class SimpleScheduleDraft(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    date: Date = Field(index=True, nullable=False)

class SimpleCalendarEvent(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    summary: Optional[str] = Field(default=None)
    schedule_draft_id: Optional[int] = Field(default=None, foreign_key="simplescheduledraft.id", index=True)

# Create engine and tables
engine = create_engine("sqlite:///:memory:", echo=False)
SQLModel.metadata.create_all(engine)

print("🧪 Testing basic foreign key functionality...")

with Session(engine) as session:
    # Create a draft
    draft = SimpleScheduleDraft(date=Date.today())
    session.add(draft)
    session.commit()
    session.refresh(draft)
    print(f"✅ Draft created: ID={draft.id}")
    
    # Create an event linked to the draft
    event = SimpleCalendarEvent(
        summary="Test Event",
        schedule_draft_id=draft.id
    )
    session.add(event)
    session.commit()
    session.refresh(event)
    print(f"✅ Event created: ID={event.id}, linked to draft {event.schedule_draft_id}")
    
    # Query events by draft
    events = session.exec(
        select(SimpleCalendarEvent).where(SimpleCalendarEvent.schedule_draft_id == draft.id)
    ).all()
    print(f"✅ Found {len(events)} events for draft {draft.id}")

print("\n🎉 BASIC FOREIGN KEY FUNCTIONALITY WORKS!")
print("💡 Next: Test the actual models with relationships after kernel restart.")

🧪 Testing basic foreign key functionality...
✅ Draft created: ID=1
✅ Event created: ID=1, linked to draft 1
✅ Found 1 events for draft 1

🎉 BASIC FOREIGN KEY FUNCTIONALITY WORKS!
💡 Next: Test the actual models with relationships after kernel restart.


In [1]:
# Test actual models with relationships - RESTART KERNEL FIRST!

print("🚀 Testing actual models with simplified relationships...")

# Import with clear imports
from datetime import date as Date
from sqlmodel import SQLModel, create_engine, Session, select
# import sys

# # Clear any cached modules first
# modules_to_clear = [name for name in sys.modules.keys() if 'fateforger.agents.schedular.models' in name]
# for mod in modules_to_clear:
#     del sys.modules[mod]

# Clear SQLModel metadata
SQLModel.metadata.clear()

# Now import our models
from fateforger.agents.schedular.models.calendar_event import CalendarEvent
from fateforger.agents.schedular.models.schedule_draft import ScheduleDraft, DraftStore

# Create engine and tables
engine = create_engine("sqlite:///:memory:", echo=False)
SQLModel.metadata.create_all(engine)

print("✅ Models imported and tables created")

with Session(engine) as session:
    store = DraftStore(session)
    
    print("\n🏗️ Testing basic CRUD operations...")
    
    # Create a draft
    draft = ScheduleDraft(date=Date.today())
    saved_draft = store.save(draft)
    print(f"✅ Draft saved: ID={saved_draft.id}")
    
    # Create events manually
    event1 = CalendarEvent(summary="Morning Meeting", schedule_draft_id=saved_draft.id)
    event2 = CalendarEvent(summary="Afternoon Review", schedule_draft_id=saved_draft.id)
    
    session.add(event1)
    session.add(event2)
    session.commit()
    session.refresh(event1)
    session.refresh(event2)
    
    print(f"✅ Events created: {event1.id}, {event2.id}")
    
    # Query events by foreign key
    events = session.exec(
        select(CalendarEvent).where(CalendarEvent.schedule_draft_id == saved_draft.id)
    ).all()
    print(f"✅ Found {len(events)} events for draft {saved_draft.id}")
    
    # Test cascade delete
    store.delete(saved_draft.id)
    
    # Verify cascade worked
    remaining_events = session.exec(select(CalendarEvent)).all()
    print(f"✅ Remaining events after cascade delete: {len(remaining_events)}")

print("\n🎉 RELATIONAL MODEL WORKS!")
print("✨ Foreign keys and cascade delete fully functional!")

🚀 Testing actual models with simplified relationships...


✅ Models imported and tables created

🏗️ Testing basic CRUD operations...
✅ Draft saved: ID=1
✅ Events created: 1, 2
✅ Found 2 events for draft 1
✅ Remaining events after cascade delete: 0

🎉 RELATIONAL MODEL WORKS!
✨ Foreign keys and cascade delete fully functional!


# Testing with a real schedule

In [ ]:

# parse_obj_as(list[CalendarEvent], CalendarEvent.model_validate_json(timebox_json))

NameError: name 'CalendarEvent' is not defined

# storing that draft

In [3]:
timebox_json = '''
[
  {
    "summary": "Sleep",
    "start": {
      "dateTime": "2025-07-29T23:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T07:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "2"
  },
  {
    "summary": "Morning routine",
    "start": {
      "dateTime": "2025-07-30T07:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T08:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "1"
  },
  {
    "summary": "Drop off dog",
    "start": {
      "dateTime": "2025-07-30T08:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T08:45:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Cycle to Gerimedica",
    "start": {
      "dateTime": "2025-07-30T08:45:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T09:05:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Shallow work (emails, admin)",
    "start": {
      "dateTime": "2025-07-30T09:05:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T09:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "8"
  },
  {
    "summary": "Deep work (C2T)",
    "start": {
      "dateTime": "2025-07-30T09:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T11:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "9"
  },
  {
    "summary": "Buffer break",
    "start": {
      "dateTime": "2025-07-30T11:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T11:45:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Shallow work",
    "start": {
      "dateTime": "2025-07-30T11:45:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T12:15:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "8"
  },
  {
    "summary": "Lunch",
    "start": {
      "dateTime": "2025-07-30T12:15:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T12:35:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Meditation",
    "start": {
      "dateTime": "2025-07-30T12:35:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T12:45:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "7"
  },
  {
    "summary": "Deep work (C2T)",
    "start": {
      "dateTime": "2025-07-30T12:45:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T14:15:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "9"
  },
  {
    "summary": "Rejuvenation break (sci-fi reading + stretch)",
    "start": {
      "dateTime": "2025-07-30T14:15:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T14:35:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Buffer break",
    "start": {
      "dateTime": "2025-07-30T14:35:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T15:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Deep work (C2T follow-up)",
    "start": {
      "dateTime": "2025-07-30T15:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T17:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "9"
  },
  {
    "summary": "Pick up dog and cycle home",
    "start": {
      "dateTime": "2025-07-30T17:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T17:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Buffer / prep",
    "start": {
      "dateTime": "2025-07-30T17:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T18:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Eat oats",
    "start": {
      "dateTime": "2025-07-30T18:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T18:10:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "TickTick deep-clean",
    "start": {
      "dateTime": "2025-07-30T18:10:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T18:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "8"
  },
  {
    "summary": "Planning session",
    "start": {
      "dateTime": "2025-07-30T18:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T18:40:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "10"
  },
  {
    "summary": "Buffer / prep for training",
    "start": {
      "dateTime": "2025-07-30T18:40:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T19:40:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Cycle to hockey",
    "start": {
      "dateTime": "2025-07-30T19:40:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T20:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Hockey training",
    "start": {
      "dateTime": "2025-07-30T20:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T21:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "6"
  },
  {
    "summary": "Cycle home",
    "start": {
      "dateTime": "2025-07-30T21:30:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T21:50:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Quick wind-down",
    "start": {
      "dateTime": "2025-07-30T21:50:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T22:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "5"
  },
  {
    "summary": "Dog walk",
    "start": {
      "dateTime": "2025-07-30T22:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T22:10:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "3"
  },
  {
    "summary": "Pack",
    "start": {
      "dateTime": "2025-07-30T22:10:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T22:15:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "3"
  },
  {
    "summary": "Hygiene",
    "start": {
      "dateTime": "2025-07-30T22:15:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T22:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "3"
  },
  {
    "summary": "Reading",
    "start": {
      "dateTime": "2025-07-30T22:25:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T22:55:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "3"
  },
  {
    "summary": "Final prep",
    "start": {
      "dateTime": "2025-07-30T22:55:00",
      "timeZone": "Europe/Amsterdam"
    },
    "end": {
      "dateTime": "2025-07-30T23:00:00",
      "timeZone": "Europe/Amsterdam"
    },
    "colorId": "3"
  }
]
'''



In [4]:
from datetime import date as Date
from sqlmodel import SQLModel, create_engine, Session, select
# import sys

# # Clear any cached modules first
# modules_to_clear = [name for name in sys.modules.keys() if 'fateforger.agents.schedular.models' in name]
# for mod in modules_to_clear:
#     del sys.modules[mod]

# Clear SQLModel metadata
SQLModel.metadata.clear()

# Now import our models
from fateforger.agents.schedular.models.calendar_event import CalendarEvent
from fateforger.agents.schedular.models.schedule_draft import ScheduleDraft, DraftStore

from pydantic import TypeAdapter
from typing import List

# Create engine and tables
engine = create_engine("sqlite:///:memory:", echo=False)
SQLModel.metadata.create_all(engine)

print("✅ Models imported and tables created")

with Session(engine) as session:
    store = DraftStore(session)
    
    print("\n🏗️ Testing basic CRUD operations...")
    
    # Create a draft
    draft = ScheduleDraft(
        date=Date.today(),
        events = TypeAdapter(List[CalendarEvent]).validate_json(timebox_json)
        )
    saved_draft = store.save(draft)

✅ Models imported and tables created

🏗️ Testing basic CRUD operations...


# Committing that draft to google calendar

In [5]:
draft.events

DetachedInstanceError: Parent instance <ScheduleDraft at 0x117491e90> is not bound to a Session; lazy load operation of attribute 'events' cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)

In [1]:
from isodate import parse_duration
from datetime import datetime

# Parse an ISO-8601 duration
delta = parse_duration("PT90M")        # returns datetime.timedelta(seconds=5400)

# Anchor at a known time
start = datetime(2025, 7, 30, 9, 0)    # July 30, 2025 @ 09:00
end   = start + delta                  # → 2025-07-30 10:30

print(end)  # 2025-07-30 10:30:00

2025-07-30 10:30:00
